In [1]:
# suppress qiskit 1.0 deprecation warnings:
import warnings
import numpy as np

# Suppress all warnings
warnings.filterwarnings("ignore")

# Example code that raises warnings
warnings.warn("This is a warning!")

from qiskit import QuantumCircuit, Aer, execute
import src.qcm_sim as sim
import src.compile_keys as keys

# import circuits:
import circuits.hidden_shift as hshift 

In [2]:
m = 3
n = 2*m
c = int(m/3)

# Generate a random hidden shift
shift = [1 for _ in range(n)]

# create qiskit circuit
qc = hshift.hidden_shift(n,c,shift,is_clifford = False)
qc_qasm = qc.qasm()

print(qc)

     ┌───┐                                                                 »
q_0: ┤ H ├───────■───────────────────────────────────────■─────────────────»
     ├───┤       │                                       │            ┌───┐»
q_1: ┤ H ├───────┼────■─────────■────────────────────────┼─────────■──┤ S ├»
     ├───┤       │    │       ┌─┴─┐┌───┐┌───┐┌───┐┌───┐┌─┴─┐┌───┐┌─┴─┐├───┤»
q_2: ┤ H ├───────┼────┼────■──┤ X ├┤ S ├┤ S ├┤ S ├┤ T ├┤ X ├┤ T ├┤ X ├┤ S ├»
     ├───┤┌───┐┌─┴─┐  │    │  ├───┤├───┤├───┤└───┘└───┘└───┘└───┘└───┘└───┘»
q_3: ┤ H ├┤ H ├┤ X ├──┼────┼──┤ H ├┤ H ├┤ H ├──────────────────────────────»
     ├───┤├───┤└───┘┌─┴─┐  │  ├───┤├───┤├───┤                              »
q_4: ┤ H ├┤ H ├─────┤ X ├──┼──┤ H ├┤ H ├┤ H ├──────────────────────────────»
     ├───┤├───┤     └───┘┌─┴─┐├───┤├───┤├───┤                              »
q_5: ┤ H ├┤ H ├──────────┤ X ├┤ H ├┤ H ├┤ H ├──────────────────────────────»
     └───┘└───┘          └───┘└───┘└───┘└───┘                              »

In [3]:
# Qiskit simulation
simulator = Aer.get_backend('qasm_simulator')
job = execute(qc, simulator, shots=1024)
result = job.result()
counts = result.get_counts()

counts

{'111111': 1024}

In [5]:
# number of samples:
number_of_t_gates = 2*7*c
epsilon = 0.1
prob_fail = 0.01
hoeffding_samples = keys.compute_hoeffding_samples(number_of_t_gates,epsilon,prob_fail)

hoeffding_samples

1025140

In [7]:
# Shift string
print(f"\nHidden Shift: {shift}\n")

# QCM simulation
results = sim.run_qcm(qc_qasm,shots=hoeffding_samples)


Hidden Shift: [1, 1, 1, 1, 1, 1]

------------------------------
Initial conditions for simulation:
------------------------------

Negativity:
 31.103388466421812

Number of shots:
 1025140

------------------------------
Outputs of Simulation:
------------------------------

Average time per shot: 0.004978 seconds

Distinct Outcome: (1, 1, 1, 0, 0, 0)
Born Rule Estimate: 0.005157906275525009

Distinct Outcome: (1, 1, 1, 1, 0, 1)
Born Rule Estimate: -0.009102187545044134

Distinct Outcome: (1, 1, 1, 1, 1, 0)
Born Rule Estimate: 0.021905931358406214

Distinct Outcome: (1, 1, 1, 0, 1, 1)
Born Rule Estimate: 0.0016383937581079442

Distinct Outcome: (1, 1, 1, 1, 0, 0)
Born Rule Estimate: 0.014684862572671204

Distinct Outcome: (1, 1, 1, 0, 0, 1)
Born Rule Estimate: 0.0017597562587085326

Distinct Outcome: (1, 1, 1, 0, 1, 0)
Born Rule Estimate: 0.0099820656743984

Distinct Outcome: (1, 1, 1, 1, 1, 1)
Born Rule Estimate: 1.0040016268435181

